In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

conn = sqlite3.connect('shotchart/shot.db')

plays = pd.read_sql('SELECT * FROM pbp_raw', conn)

plays.head(10)

/home/kieran/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


,game_id,quarter,time_remaining,away_play,away_pts,score,home_pts,home_play
0,202012230IND,1,<td>11:40.0</td>,<td> </td>,<td> </td>,"<td class="" center"">0-0</td>","<td class=""""> </td>","<td class="""">Turnover by <a href=""/players/s/s..."
1,202012230IND,1,<td>11:19.0</td>,"<td class=""""><a href=""/players/r/randlju01.htm...","<td class=""""> </td>","<td class="" center"">0-0</td>",<td> </td>,<td> </td>
2,202012230IND,1,<td>11:16.0</td>,<td> </td>,<td> </td>,"<td class="" center"">0-0</td>","<td class=""""> </td>","<td class="""">Defensive rebound by <a href=""/pl..."
3,202012230IND,1,<td>11:05.0</td>,<td> </td>,<td> </td>,"<td class="" center"">0-0</td>","<td class=""""> </td>","<td class=""""><a href=""/players/t/turnemy01.htm..."
4,202012230IND,1,<td>11:01.0</td>,"<td class="""">Defensive rebound by <a href=""/pl...","<td class=""""> </td>","<td class="" center"">0-0</td>",<td> </td>,<td> </td>
5,202012230IND,1,<td>10:58.0</td>,"<td class="""">Offensive foul by <a href=""/playe...","<td class=""""> </td>","<td class="" center"">0-0</td>",<td> </td>,<td> </td>
6,202012230IND,1,<td>10:58.0</td>,"<td class="""">Turnover by <a href=""/players/b/b...","<td class=""""> </td>","<td class="" center"">0-0</td>",<td> </td>,<td> </td>
7,202012230IND,1,<td>10:45.0</td>,<td> </td>,<td> </td>,"<td class="" center"">0-0</td>","<td class=""""> </td>","<td class=""""><a href=""/players/w/warretj01.htm..."
8,202012230IND,1,<td>10:41.0</td>,"<td class="""">Defensive rebound by <a href=""/pl...","<td class=""""> </td>","<td class="" center"">0-0</td>",<td> </td>,<td> </td>
9,202012230IND,1,<td>10:36.0</td>,"<td class=""""><a href=""/players/r/randlju01.htm...","<td class=""""> </td>","<td class="" center"">0-0</td>",<td> </td>,<td> </td>


In [2]:
def get_play(row):
    home = row['home_play']
    away = row['away_play']
    
    if home == '<td>\xa0</td>':
        return pd.Series({'home' : False, 'p_text' : away})
    elif away == '<td>\xa0</td>':
        return pd.Series({'home' : True, 'p_text' : home})
    
p_texts = plays.progress_apply(get_play, axis = 1)

100%|██████████| 406207/406207 [02:09<00:00, 3125.61it/s]


In [3]:
players = r'href="/players/[a-z]/(\w+)'
p_texts.loc[:, 'Players'] = p_texts['p_text'].str.findall(players)
p_texts

,home,p_text,Players
0,True,"<td class="""">Turnover by <a href=""/players/s/s...","[sabondo01, bullore01]"
1,False,"<td class=""""><a href=""/players/r/randlju01.htm...",[randlju01]
2,True,"<td class="""">Defensive rebound by <a href=""/pl...",[brogdma01]
3,True,"<td class=""""><a href=""/players/t/turnemy01.htm...",[turnemy01]
4,False,"<td class="""">Defensive rebound by <a href=""/pl...",[robinmi01]
...,...,...,...
406202,True,"<td class="""">Personal take foul by <a href=""/p...","[dragigo01, fournev01]"
406203,True,"<td class=""bbr-play-score""><a href=""/players/f...",[fournev01]
406204,True,"<td class=""""><a href=""/players/f/fournev01.htm...",[fournev01]
406205,False,"<td class="""">Defensive rebound by <a href=""/pl...",[robindu01]


In [4]:
subs = p_texts[p_texts['p_text'].str.contains(' enters ')]
subs = pd.concat([plays.iloc[subs.index], subs], axis = 1)
subs.loc[:, 'enters'] = subs['Players'].str[0]
subs.loc[:, 'leaves'] = subs['Players'].str[1]

sub_times = subs['time_remaining'].str.extract('(\d+):(\d{2}).(\d)')
sub_times.columns = ['Minutes', 'Seconds', 'Decimal']
sub_times = sub_times.astype(int)
sub_times.loc[:, 'total_time'] = 60 * sub_times['Minutes'] + sub_times['Seconds'] + sub_times['Decimal']/10

subs = pd.concat([subs, sub_times['total_time']], axis = 1)
subs[['game_id', 'quarter', 'home', 'enters', 'leaves', 'total_time']]

,game_id,quarter,home,enters,leaves,total_time
34,202012230IND,1,False,noelne01,robinmi01,445.0
41,202012230IND,1,True,holidju01,turnemy01,389.0
59,202012230IND,1,False,burksal01,bullore01,289.0
60,202012230IND,1,True,mcderdo01,warretj01,289.0
77,202012230IND,1,False,quickim01,paytoel01,207.0
...,...,...,...,...,...,...
406172,202012230ORL,4,False,dragigo01,herroty01,140.0
406173,202012230ORL,4,False,iguodan01,olynyke01,140.0
406192,202012230ORL,4,False,herroty01,robindu01,28.0
406195,202012230ORL,4,False,robindu01,iguodan01,28.0


## Subs

| Time | Enters | Exits |
|:---|----|----:|
|12:00|p1||
|12:00|p2||
|12:00|p3||
|6:05|p2|p4|
|0:00||p1|
|0:00||p3|
|0:00||p4|

Becomes: 

|Player|Entry Time|Exit Time|
|:---|----|---:|
|p1|12:00|0:00|
|p2|12:00|6:05|
|p3|12:00|0:00|
|p4|6:05|0:00|

In [5]:
def get_sub_order(sub_inds):
    '''expects a df of all a teams substitutions in a given quarter'''
    inds = sub_inds.index
    order = subs.loc[inds].sort_values('total_time', ascending = False)
    players = set(order['enters'].tolist() + order['leaves'].tolist())
    
    game = subs.loc[inds, 'game_id'].unique()[0]
    quarter = subs.loc[inds, 'quarter'].unique()[0]
    home = subs.loc[inds, 'home'].unique()[0]
    
    all_subs = pd.DataFrame()
    
    for player in players:
        entries = order.query('enters == @player')['total_time'].tolist()
        leaves = order.query('leaves == @player')['total_time'].tolist()
        
        if len(entries) > len(leaves):
            # add a row to leaves at end of quarter
            leaves.append(0)
        elif len(entries) < len(leaves):
            # add a row to entries at start of quarter
            entries.insert(0, 60 * 12)
        if min(entries) < min(leaves):
            # add a row to leaves at end of quarter
            leaves.append(0)
        if max(entries) < max(leaves):
            # add a row to entries at start of quarter
            entries.insert(0, 60 * 12)
        if len(entries) > len(leaves):
            # add a row to leaves at end of quarter
            leaves.append(0)
        elif len(entries) < len(leaves):
            # add a row to entries at start of quarter
            entries.insert(0, 60 * 12)
        
        
        assert len(entries) == len(leaves), (game, quarter, player, entries, leaves)
        assert all([x >= y for x, y in zip(entries, leaves)]), (game, quarter, player, entries, leaves)
        
        player_subs = pd.DataFrame({'Player_ID' : player, 'Home' : home, 'Game_ID' : game, 'Quarter' : int(quarter), 'Home' : home, 
                                   'Entry_Time' : entries, 'Exit_Time' : leaves})
        all_subs = all_subs.append(player_subs)
        
    return all_subs    

ch_subs = subs.query('game_id == "202101010CHO"').groupby(['quarter', 'home']).apply(get_sub_order).reset_index(drop = True)

In [6]:
def get_oncourt(time, quarter, subs):
    if time == 0:
        return subs.query('Exit_Time == 0 & Quarter == @quarter')
    on = subs.query('Entry_Time >= @time & Exit_Time < @time & Quarter == @quarter')
    
    return on.sort_values(['Home', 'Entry_Time'])

get_oncourt(1, 2, ch_subs)

,Player_ID,Home,Game_ID,Quarter,Entry_Time,Exit_Time
31,anderky01,False,202101010CHO,2,74.0,0.0
26,jonesty01,False,202101010CHO,2,179.0,0.0
20,dienggo01,False,202101010CHO,2,208.0,0.0
24,konchjo01,False,202101010CHO,2,267.0,0.0
21,brookdi01,False,202101010CHO,2,408.0,0.0
40,biyombi01,True,202101010CHO,2,38.0,0.0
36,grahade01,True,202101010CHO,2,179.0,0.0
38,haywago01,True,202101010CHO,2,179.0,0.0
34,roziete01,True,202101010CHO,2,408.0,0.0
41,bridgmi02,True,202101010CHO,2,408.0,0.0


In [7]:
all_subs = subs.groupby(['game_id', 'quarter', 'home']).progress_apply(get_sub_order).reset_index(drop = True)

100%|██████████| 7246/7246 [04:55<00:00, 24.55it/s]


In [8]:
len(all_subs)

74005

In [9]:
cursor = conn.cursor()
cursor.execute('DROP TABLE subs')
conn.commit()

In [10]:
all_subs.to_sql('subs', conn, index = False)